In [8]:
from datetime import datetime
from rich import print
import pandas as pd
import json
import re

# Display all columns and rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
# Compile the regular expression for efficiency
pattern_span = re.compile(r'<span class=\"highlight\">(.*?)</span>')
pattern_spaces = re.compile(r'\s+')

def clean_span_tags(text):
    # First, remove the span tags
    cleaned_text = pattern_span.sub(r'\1', text)
    
    # Then, replace multiple spaces with a single space
    cleaned_text = pattern_spaces.sub(' ', cleaned_text)
    
    return cleaned_text

def clean_data(data):
    if isinstance(data, dict):
        return {key: clean_data(value) for key, value in data.items()}
    elif isinstance(data, list):
        return [clean_data(item) for item in data]
    elif isinstance(data, str):
        return clean_span_tags(data)
    else:
        return data

# Load the JSON data
with open('../data/raw/nasa_techtransfer_data.json', 'r') as file:
    data = json.load(file)

# Clean the data
interm_data = clean_data(data)

# Specify the directory path where you want to save the file
directory_path = r'C:\Users\640124\Documents\Code\Dev-Code\Nasa-TechTransfer\data\interm'

# Specify the filename
filename = 'interm_nasa_techtransfer_data.json'

# Combine the directory path and filename
full_file_path = f'{directory_path}/{filename}'

# Save the cleaned data back to a JSON file
with open(full_file_path, 'w') as file:
    json.dump(interm_data, file, indent=4)

In [ ]:
df = pd.read_json('../data/interm/interm_nasa_techtransfer_data.json')

In [ ]:
df.info()

In [ ]:
df.head(10)

### Column Structure 

```
Col Index 0: id
Col Index 1: ref_id
Col Index 2: project_name
Col Index 3: project_overview
Col Index 4: ref_id_2 (might drop)
Col Index 5: category
Col Index 6: release_type
Col Index 7: notes
Col Index 8: download_link
Col Index 9: agency
Col Index 10: unused_col_1
Col Index 11: unused_col_2
Col Index 12: unused_col_3
```

In [ ]:
# Example: new column names list
new_column_names = [
    "id",
    "ref_id",
    "project_name",
    "project_overview",
    "ref_id_2",
    "category",
    "release_type",
    "notes",
    "download_link",
    "agency",
    "unused_col_1",
    "unused_col_2",
    "unused_col_3"
]

# Creating a dictionary: current index to new name
rename_dict = {i: new_name for i, new_name in enumerate(new_column_names)}

# Renaming the columns
df.rename(columns=rename_dict, inplace=True)

In [ ]:
df.head(10)

In [ ]:
# Replace the empty strings with NaN in the unused columns
df.replace('', pd.NA, inplace=True)

In [ ]:
# Get the number of missing values in each column
missing_values = df.isnull().sum()

In [ ]:
print(missing_values)

In [ ]:
# Drop the unused columns
df.drop(columns=['ref_id_2', 'unused_col_1', 'unused_col_2', 'unused_col_3'], inplace=True)

# Add a new column named date_extracted with the current date
df['date_extracted'] = datetime.now().strftime('%Y-%m-%d')

In [ ]:
# Create 'github_repo' column with GitHub links, else NaN
df['github_repo'] = df['download_link'].apply(lambda x: x if pd.notna(x) and 'github' in x else pd.NA)

In [ ]:
df.head(10)

In [ ]:
# Replace 'download_link' instances containing 'github' with NA
df['download_link'] = df['download_link'].apply(lambda x: pd.NA if pd.notna(x) and 'github' in x else x)

In [ ]:
df.head(10)

In [ ]:
# Now I want to re-arange the columns
df = df[['id', 'ref_id', 'agency', 'project_name', 
        'project_overview', 'category', 'release_type', 
        'download_link', 'github_repo', 'notes', 'date_extracted']]

# Drop duplicate rows based on all columns
df.drop_duplicates(inplace=True)

In [ ]:
df.head(20)

In [ ]:
df['download_link'].unique()

In [ ]:
df.query('download_link == "For questions, help requests, bug reports, etc., please contact John L. Bresina at john.l.bresina@nasa.gov."')

### Testing out the saved json data from the transform directory

In [9]:
df = pd.read_csv('../data/processed/processed_nasa_techtransfer_data.csv')

In [10]:
# Transform the dataframe into json format
data = df.to_json(orient='records', lines=True, indent=4)

In [11]:
print(data)

{
        "id":"57f73311ab4c7425039aa318",
        "ref_id":"NPO-45583-1",
        "agency":"JPL",
        "project_name":"Autonomous Data Reduction for NASA Earth Science Data",
        "project_overview":"NASA atmospheric and Earth science missions generate vast amounts of data every day. 
Clustering algorithms are used in various applications including pattern recognition, classification, data 
compression, regression, and related optimization problems. The design implements a computational solution for an 
autonomous data-reduction\/clustering process to produce a representative distribution and joint relationships of 
the data without assuming a specific type of distribution and relationship nor resorting to domain-specific 
knowledge about the data. ",
        "category":"data and image processing",
        "release_type":"U.S. Government Purpose Release",
        "download_link":null,
        "github_repo":null,
        "notes":"https:\/\/download.jpl.nasa.gov\/ops\/request\/index.cfm, request software",
        "date_extracted":"2024-07-16"
    }

    {
        "id":"6260ee611c01d114a968bc77",
        "ref_id":"NPO-50897-1",
        "agency":"JPL",
        "project_name":"OIIP Data Viewer",
        "project_overview":"The Oceanographic In-Situ Interoperability Project aims to integrate diverse in-situ 
datasets with related remote sensing products from NASA. The data viewer is the web application through which users
can explore and interrogate the range of data available within this project. The application will reveal the 
multi-dimensional facets of in-situ data at full data resolution. It provides an coherent search interface that 
accommodates the heterogeneity of in-situ products and sophisticated charting options that provide granular control
over data displays.",
        "category":"data and image processing",
        "release_type":"Open Source",
        "download_link":null,
        "github_repo":"https:\/\/github.com\/oiip\/oiip-data-viewer",
        "notes":null,
        "date_extracted":"2024-07-16"
    }

    {
        "id":"6038973abea502310e769934",
        "ref_id":"MFS-31700-1",
        "agency":"MSFC",
        "project_name":"High-Speed Data Viewer",
        "project_overview":"The High-Speed Data Viewer computer software was developed for viewing high-frequency 
data recorded in the East and West Test Areas at Marshall Space Flight Center (MSFC).Features include: Amplitude 
vs. Time Plots with full zoom capabilities; Frequency Component Plot at specified time interval; Waterfall 
Plots\/Frequency Intensity Plots display changing frequency components over a duration of time; print options for 
plots; export functions save plot data as text files for importing into other applications. This software allows 
the user to plot data in time domain and view frequency components in multiple ways.",
        "category":"system testing",
        "release_type":"U.S. Release Only",
        "download_link":null,
        "github_repo":null,
        "notes":null,
        "date_extracted":"2024-07-16"
    }

    {
        "id":"582565a4ab4c74af2e478865",
        "ref_id":"NPO-50157-1",
        "agency":"JPL",
        "project_name":"NEXUS: Deep Data Platform",
        "project_overview":"NEXUS provides a bridge between science data and horizontal-scaling data analysis. It 
provides a workflow to divide science artifacts into small data tiles to store in a cloud-scaled database where 
they can be quickly retrieved through a high performance spatial search registry. Through spatial index, NEXUS 
provides fast access to all relevant data tiles where each tile fits into RAM for fast in-memory computation. ",
        "category":"data servers processing and handling",
        "release_type":"Open Source",
        "download_link":null,
        "github_repo":"https:\/\/github.com\/apache\/incubator-sdap-nexus",
        "notes":"https:\/\/download.jpl.nasa.gov\/ops\/request\/index.cfm, request software",
        "date_extract